In [2]:
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d

In [3]:
# Part 1: Data Setup and Yield Curve Interpolation----

def create_yield_curve_interpolator(maturities_years, rates_percent):
    rates_decimal = np.array(rates_percent) / 100

    interpolator = interp1d(maturities_years, rates_decimal, kind='linear', fill_value="extrapolate")
    print(f"Yield curve interpolator created for maturities: {maturities_years} years")
    print(f"With corresponding rates: {rates_percent}%")
    return interpolator

In [4]:
# Hypothetical market yield curve data (Years, Annual Rate in %)
# You would replace this with actual market data or the data you used previously.
market_maturities = np.array([0.5, 1.0, 2.0, 3.0, 4.0, 5.0, 7.0, 10.0, 15.0, 20.0])
market_rates = np.array([6.0, 6.2, 6.5, 6.7, 6.9, 7.0, 7.2, 7.5, 7.8, 8.0])

In [5]:
# Create the interpolator
yield_curve_interpolator = create_yield_curve_interpolator(market_maturities, market_rates)

Yield curve interpolator created for maturities: [ 0.5  1.   2.   3.   4.   5.   7.  10.  15.  20. ] years
With corresponding rates: [6.  6.2 6.5 6.7 6.9 7.  7.2 7.5 7.8 8. ]%


In [6]:
#--Bond Parameters
PRINCIPAL = 10_00_000 #-10 Lakh
COUPON_RATE_ANNUAL = 0.08
MATURITY_YEARS = 10
COMPOUNDING_FREQUENCY = 2

In [7]:
# Calculate Semi-annual coupon rate and number of periods
COUPON_RATE_SEMI_ANNUAL = COUPON_RATE_ANNUAL / COMPOUNDING_FREQUENCY
NUM_PERIODS = MATURITY_YEARS * COMPOUNDING_FREQUENCY
SEMI_ANNUAL_COUPON_PAYMENT = PRINCIPAL * COUPON_RATE_SEMI_ANNUAL

In [8]:
print(f"\nBond Parameters:")
print(f" Principal: Rs. {PRINCIPAL:,.2f}")
print(f" Annual Coupon Rate: {COUPON_RATE_ANNUAL*100}%")
print(f" Maturity: {MATURITY_YEARS} years")
print(f" Compounding Frequency: {COMPOUNDING_FREQUENCY} (Semi-Annually)")
print(f" Semi-annual Coupon Payment: Rs. {SEMI_ANNUAL_COUPON_PAYMENT:,.2f}")
print(f" Total Number of Periods: {NUM_PERIODS}")


Bond Parameters:
 Principal: Rs. 1,000,000.00
 Annual Coupon Rate: 8.0%
 Maturity: 10 years
 Compounding Frequency: 2 (Semi-Annually)
 Semi-annual Coupon Payment: Rs. 40,000.00
 Total Number of Periods: 20


In [9]:
#--Part 2: Bond Cash flow Generation--
def generate_bond_cash_flows(principal, coupon_payment_per_period, num_periods):
    cash_flows = []
    time_to_cash_flow_years = []

    for i in range(1, num_periods + 1):
        # Time in years for semi-annual payments (e.g., 0.5, 1.0, 1.5, ...)
        time_in_years = i / COMPOUNDING_FREQUENCY

        if i == num_periods:
            # Last cash flow includes principal repayment
            cash_flows.append(coupon_payment_per_period + principal)

        else:
            cash_flows.append(coupon_payment_per_period)
        time_to_cash_flow_years.append(time_in_years)

    print(f"\nGenerated {len(cash_flows)} cash flows.")

    return np.array(cash_flows), np.array(time_to_cash_flow_years)

cash_flows, time_to_cash_flow_years = generate_bond_cash_flows(
    PRINCIPAL, SEMI_ANNUAL_COUPON_PAYMENT, NUM_PERIODS
)
            
        
        


Generated 20 cash flows.


In [10]:
#--- Part 3: Bond Pricing Function---
def calculate_bond_price(cash_flows, time_to_cash_flow_years, ytm_annual_decimal, compounding_frequency):
     # Convert annual YTM to YTM per compounding period
    ytm_per_period = ytm_annual_decimal / compounding_frequency

    present_values = cash_flows / ((1 + ytm_per_period)**(time_to_cash_flow_years * compounding_frequency))
    bond_price = np.sum(present_values)

In [33]:
#--- Part 4: Duration Calcualation---

def calculate_macaulay_duration(cash_flow, time_to_cash_flow_years, bond_price, ytm_annual_decimal, compounding_frequency):
    if bond_price == 0:
        return 0.0 #avoid division by zero

    ytm_per_period = ytm_annual_decimal / compounding_frequency

    #Calcualte Present value of each cash flow
    pv_cash_flows = cash_flow / ((1 + ytm_per_period) ** (time_to_cash_flow_years * compounding_frequency))

    #Calcuate weighted time to cash flow
    weighted_times = (pv_cash_flows / bond_price) * time_to_cash_flow_years

    macaulay_duration = np.sum(weighted_times)
    return macaulay_duration

In [12]:
def calculate_modified_duration(macaulay_duration, ytm_annual_decimal, compounding_frequency):
    modified_duration = macaulay_duration / (1 + (ytm_annual_decimal / compounding_frequency))
    return modified_duration

In [13]:
#--Part 5:- DV01 Calculations
def calculate_dv01(modified_duration, bond_price):
    dv01 = modified_duration * bond_price * 0.0001
    return dv01


In [21]:
#-- Part 6:- Application  and Analysis--
print("\n--- Bond Analysis-----")
ytm_for_10_years = yield_curve_interpolator(MATURITY_YEARS)
print(f"\nInterpolated YTM for {MATURITY_YEARS} years: {ytm_for_10_years*100:.4f}%")
ytm_for_10_years = ytm_for_10_years.item()


--- Bond Analysis-----

Interpolated YTM for 10 years: 7.5000%


In [31]:
# Calculate the bond price using this YTM
bond_price_at_10_years_ytm = calculate_bond_price(
    cash_flows, time_to_cash_flow_years, ytm_for_10_years, COMPOUNDING_FREQUENCY
)

print(f"Bond Price (at {ytm_for_10_years*100:.4f}% YTM): Rs. {bond_price_at_10_years_ytm:,.2f}")

Bond Price (at 7.5000% YTM): Rs. 1,034,740.51


In [30]:
ytm_for_10_years = yield_curve_interpolator(MATURITY_YEARS)
ytm_for_10_years = float(ytm_for_10_years)  # Convert array to float

bond_price_at_10_years_ytm = calculate_bond_price(
    cash_flows, time_to_cash_flow_years, ytm_for_10_years, COMPOUNDING_FREQUENCY
)

print(f"Bond Price (at {ytm_for_10_years*100:.4f}% YTM): Rs.{bond_price_at_10_years_ytm:.2f}")


Bond Price (at 7.5000% YTM): Rs.1034740.51


In [24]:
print("YTM Output:", ytm_for_10_years)
print("Type:", type(ytm_for_10_years))



YTM Output: 0.075
Type: <class 'float'>


In [26]:
bond_price_at_10_years_ytm

In [28]:
def calculate_bond_price(cash_flows, time_to_cash_flow_years, ytm, compounding_frequency):
    bond_price = 0
    for cf, t in zip(cash_flows, time_to_cash_flow_years):
        discount_factor = (1 + ytm / compounding_frequency) ** (compounding_frequency * t)
        bond_price += cf / discount_factor

    return bond_price  # ✅ This must be included


In [34]:
# Calculate Macaulay and Modified Duration
macaulay_dur = calculate_macaulay_duration(
    cash_flows, time_to_cash_flow_years, bond_price_at_10_years_ytm, ytm_for_10_years, COMPOUNDING_FREQUENCY
)
modified_dur = calculate_modified_duration(
    macaulay_dur, ytm_for_10_years, COMPOUNDING_FREQUENCY
)

print(f"\na. Duration of Bond (at {MATURITY_YEARS} years maturity):")
print(f"   Macaulay Duration: {macaulay_dur:.4f} years")
print(f"   Modified Duration: {modified_dur:.4f}")


a. Duration of Bond (at 10 years maturity):
   Macaulay Duration: 7.1225 years
   Modified Duration: 6.8651


In [35]:
# b. Calculate the DV01 risk of bond across 1Y, 2Y, 3Y, 4Y, 5Y, 3.5Y, 4.5Y, 5.5Y points
target_maturities_for_dv01 = [1.0, 2.0, 3.0, 4.0, 5.0, 3.5, 4.5, 5.5]
dv01_results = []

print("\nb. DV01 Risk of Bond across various yield curve points:")

for target_maturity in target_maturities_for_dv01:
    ytm_for_dv01 = yield_curve_interpolator(target_maturity)
    
    # Recalculate bond price based on this YTM
    current_bond_price = calculate_bond_price(
        cash_flows, time_to_cash_flow_years, ytm_for_dv01, COMPOUNDING_FREQUENCY
    )
    
    # Recalculate Macaulay and Modified Duration based on this YTM
    current_macaulay_dur = calculate_macaulay_duration(
        cash_flows, time_to_cash_flow_years, current_bond_price, ytm_for_dv01, COMPOUNDING_FREQUENCY
    )
    current_modified_dur = calculate_modified_duration(
        current_macaulay_dur, ytm_for_dv01, COMPOUNDING_FREQUENCY
    )
    


b. DV01 Risk of Bond across various yield curve points:


In [36]:

    # Calculate DV01
    current_dv01 = calculate_dv01(current_modified_dur, current_bond_price)
    
    dv01_results.append({
        "Target Maturity (Years)": target_maturity,
        "Interpolated YTM (%)": f"{ytm_for_dv01*100:.4f}",
        "Bond Price (Rs.)": f"{current_bond_price:,.2f}",
        "Modified Duration": f"{current_modified_dur:.4f}",
        "DV01 (Rs.)": f"{current_dv01:,.4f}"
    })

# Display results in a table
df_dv01 = pd.DataFrame(dv01_results)
print(df_dv01.to_string(index=False))

print("\n--- End of Analysis ---")


 Target Maturity (Years) Interpolated YTM (%) Bond Price (Rs.) Modified Duration DV01 (Rs.)
                     5.5               7.0500     1,067,356.46            6.9279   739.4559

--- End of Analysis ---
